In [1]:
from aiida import load_profile

load_profile()

Profile<uuid='c677ac2b24bc40178ad9f1681fc23d0a' name='presto'>

In [2]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [3]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [4]:
from aiida.orm import load_code
code = load_code("janus@localhost")

In [5]:
from aiida.orm import Float, Dict, Str

inputs = {
    "code": code,
    "model": model,
    "struct": structure,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "calc_kwargs": Dict({}),
    "min_volume": Float(0.95),
    "max_volume": Float(1.05),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
}

In [6]:
from aiida.plugins import CalculationFactory
eosCalc = CalculationFactory("mlip.eos")

In [7]:
from aiida.engine import run_get_node
result, node = run_get_node(eosCalc, inputs)

11/28/2025 01:27:04 PM <93414> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
11/28/2025 01:27:04 PM <93414> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
11/28/2025 01:27:04 PM <93414> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.


outputs <AttributesFrozendict {'code': <InstalledCode: Remote code 'janus' on localhost pk: 1, uuid: 294ec8d5-4250-4245-8040-d20377fa8233>, 'model': <ModelData: uuid: 8fb3d4e2-5d7a-4ea1-9b41-d821624e0712 (pk: 2)>, 'struct': <StructureData: uuid: d4ec68e2-b673-4a66-8e31-9a3775769388 (pk: 5900)>, 'arch': <Str: uuid: d77d54a6-806a-41a8-b5ae-70be42a9b984 (pk: 5901) value: mace_mp>, 'device': <Str: uuid: e585b4b8-d222-4397-b58f-9cd658edf6c0 (pk: 5902) value: cpu>, 'calc_kwargs': <Dict: uuid: 1aa2ac23-e0ae-4ed0-b9ad-2d37ba0775d2 (pk: 5903)>, 'min_volume': <Float: uuid: 36b07414-321b-4dfa-803c-0d60a718a428 (pk: 5904) value: 0.95>, 'max_volume': <Float: uuid: 8e89b0f6-0a21-40e6-9419-a61d31bb1c66 (pk: 5905) value: 1.05>, 'metadata': <AttributesFrozendict {'options': <AttributesFrozendict {'resources': {'num_machines': 1}, 'input_filename': 'aiida.xyz', 'output_filename': 'aiida-stdout.txt', 'submit_script_filename': '_aiidasubmit.sh', 'scheduler_stdout': '_scheduler-stdout.txt', 'scheduler_stde

11/28/2025 01:27:13 PM <93414> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [ERROR] invalid value uuid: 0ab69414-6cc1-47e2-a07a-94f7ec2c095c (unstored) specified with label aiida-eos-fit.dat: port 'aiida-eos-fit' does not exist in port namespace 'outputs'
11/28/2025 01:27:13 PM <93414> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [WARNING] output parser returned exit code<10>: The process returned an invalid output.


retrieved ['_scheduler-stderr.txt', '_scheduler-stdout.txt', 'aiida-stdout.txt', 'aiida.log', 'eos-summary.yml']
outputs_folder /home/strix/.aiida/scratch/presto/a6/e3/1ea6-6845-42da-8c01-5538a5912c0c
node ['aiida-stdout.txt', 'a6e31ea6-6845-42da-8c01-5538a5912c0c', 'aiida.log', 'eos-summary.yml', 'xyz_output', '_scheduler-stdout.txt', '_scheduler-stderr.txt']
option {'input_filename': 'aiida.xyz', 'output_filename': 'aiida-stdout.txt', 'submit_script_filename': '_aiidasubmit.sh', 'scheduler_stdout': '_scheduler-stdout.txt', 'scheduler_stderr': '_scheduler-stderr.txt', 'resources': {'num_machines': 1}, 'custom_scheduler_commands': '', 'mpirun_extra_params': [], 'import_sys_environment': True, 'environment_variables': {}, 'environment_variables_double_quotes': False, 'prepend_text': '', 'append_text': '', 'parser_name': 'mlip.eos_parser'}
aiida-generated.extxyz


In [8]:
print(result)
print("retireved", result["retrieved"].list_object_names())
print("remote dir", result["remote_folder"].listdir())
print("remote janus dir", result["remote_folder"].listdir('janus_results'))
print("remote janus dir", result["remote_folder"].listdir('janus_results'))
print(node)

{'remote_folder': <RemoteData: uuid: 9391bd22-feb1-4cef-882c-e88e8feae424 (pk: 5909)>, 'retrieved': <FolderData: uuid: 59d2860f-31c9-44fe-a24b-7a00a471e803 (pk: 5910)>, 'log_output': <SinglefileData: uuid: 6619eedd-c42d-4f58-a78d-9f8ba189b52c (pk: 5911)>, 'std_output': <SinglefileData: uuid: f4ff705b-2e2a-41f2-86df-1cadda000469 (pk: 5912)>, 'xyz_output': <SinglefileData: uuid: 8275ae40-5697-4684-8272-60e7e75ddc52 (pk: 5913)>}
retireved ['_scheduler-stderr.txt', '_scheduler-stdout.txt', 'aiida-stdout.txt', 'aiida.log', 'eos-summary.yml']
remote dir ['aiida.log', 'aiida.xyz', '_scheduler-stderr.txt', 'eos-summary.yml', '_scheduler-stdout.txt', 'janus_results', 'aiida-stdout.txt', 'mlff.model', '.aiida', '_aiidasubmit.sh']
remote janus dir ['aiida-generated.extxyz', 'aiida-eos-fit.dat', 'aiida-eos-raw.dat']
remote janus dir ['aiida-generated.extxyz', 'aiida-eos-fit.dat', 'aiida-eos-raw.dat']
uuid: a6e31ea6-6845-42da-8c01-5538a5912c0c (pk: 5908) (aiida.calculations:mlip.eos)


In [ ]:
from aiida.orm import load_node

print(load_node(5883).get_content())